# Funciones auxiliares

Cada operación básica devuelve una lista de pares (probabilidad, termino)


In [2]:
def apply_gate(gate, terms):
    def op(circuit):
        circuit = circuit.copy()
        if gate=='CNOT':
            circuit.cx(0, 1)
        elif gate=='H':
            circuit.h(0)
        elif gate=='X':
            circuit.x(0)
        else:
            raise Exception('Unknown gate')
        return circuit
    return [(t[0], op(t[1])) for t in terms]

In [3]:
# agrega una medicion sobre todos los qubits del circuito
def apply_measure(terms):
    def op(circuit):
        circuit = circuit.copy()
        qubits = list(range(circuit.num_qubits))
        circuit.measure(qubits, qubits)
        return circuit
    return [(t[0], op(t[1])) for t in terms]

In [4]:
def empty_circuit(qubits):
    return QuantumCircuit(qubits, qubits)

In [5]:
# crea |010+> a partir de '010+'
def make_pure(qubits):
    qubits = str(qubits)
    circuit = empty_circuit(len(qubits))
    for i,qubit in enumerate(qubits):
        if qubit=='0':
            pass
        elif qubit=='1':
            circuit.x(i)
        elif qubit=='+':
            circuit.h(i)
        elif qubit=='-':
            circuit.x(i)
            circuit.h(i)
        else:
            raise Exception('Unexpected qubit')
    return [(1, circuit)]

In [6]:
example = make_pure('01+-')
example[0][1].draw()

q_0: ──────────
     ┌───┐     
q_1: ┤ X ├─────
     ├───┤     
q_2: ┤ H ├─────
     ├───┤┌───┐
q_3: ┤ X ├┤ H ├
     └───┘└───┘
c: 4/══════════

In [7]:
def tensor_product(terms1, terms2):
    def op(circuit1, circuit2):
        aux = empty_circuit(circuit1.num_qubits+circuit2.num_qubits)
        circuit2_qubits = [i+circuit1.num_qubits for i in range(circuit2.num_qubits)]
        return aux.compose(circuit1).compose(circuit2, qubits=circuit2_qubits, clbits=circuit2_qubits)
    return [(t1[0]*t2[0], op(t1[1], t2[1])) for t1 in terms1 for t2 in terms2]

|01+-> $\otimes$ |01+-> = |01+-01+->

In [8]:
prod_example = tensor_product(example, example)
print(prod_example)
prod_example[0][1].draw()

[(1, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fb1a0f92e80>)]


q_0: ──────────
     ┌───┐     
q_1: ┤ X ├─────
     ├───┤     
q_2: ┤ H ├─────
     ├───┤┌───┐
q_3: ┤ X ├┤ H ├
     └───┘└───┘
q_4: ──────────
     ┌───┐     
q_5: ┤ X ├─────
     ├───┤     
q_6: ┤ H ├─────
     ├───┤┌───┐
q_7: ┤ X ├┤ H ├
     └───┘└───┘
c: 8/══════════

In [9]:
def measurement_results(terms, shots=1000):
    shots_per_term = math.ceil(float(shots)/len(terms))
    accum_probs = {}
    for prob,circuit in terms:
        job = execute(circuit, backend, shots=shots_per_term)
        counts = job.result().get_counts(circuit)
        accum_probs = {k: prob*counts.get(k, 0.0)/shots_per_term + accum_probs.get(k, 0.0)
                        for k in set(counts) | set(accum_probs)}
    return [(prob, res) for res,prob in accum_probs.items()] # returns [(1, '00101')]

In [10]:
def letcase(condition_term, cases):
    letresults = measurement_results(condition_term)
    results = []
    for prob,result in letresults:
        y = make_pure(result)
        results+=[(prob*prob2, term) for prob2,term in cases[result](y)]
    return results

# Programa para la expresión


$\text{prog} = \lambda x. \pi^2\mathbf{CNOT} ((\text{letcase } y = \pi^1Hx \text{ in }\{\text{|0>}, \text{|1>}\}) \otimes \text{|0>})$

In [11]:
prog = (lambda x:
    apply_measure(
        apply_gate('CNOT',
            tensor_product(
                letcase(
                    apply_measure(apply_gate('H', x)),
                    {
                        '0': lambda y: make_pure('0'),
                        '1': lambda y: make_pure('1'),
                    }
                ),
                make_pure('0')
            )
        )
    )
)

$\text{exp} = \text{prog}\;\text{|1>}$

In [12]:
exp = prog(make_pure('1'))

In [13]:
exp

[(0.506, <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7fb1d13a98b0>),
 (0.494, <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7fb1d13a9880>)]

In [14]:
measurement_results(exp)

[(0.506, '00'), (0.494, '11')]